In [ ]:
!pip install -U langchain-community
!pip install langchain
!pip install torch
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 70.3 M

### Saudações a todos,
#### Para esse projeto é importante ressaltar que será necessário obter uma chave gratuita do Gemini, abaixo está o passo-a-passo:
    1. Acesse o site oficial do Gemini: https://aistudio.google.com/app/apikey

    2. Crie uma conta gratuita ou faça login com uma conta Google.

    3. Vá para a seção API Keys ou Credenciais do painel do usuário.

    4. Clique em Criar nova chave ou Generate API Key.

    5. Copie a chave gerada e guarde em um local seguro.


#### RAG (Geração com Recuperação de Informações) é uma técnica que combina:

Modelos de linguagem (LLMs) – como GPT, BERT, etc., capazes de gerar texto ou responder perguntas.

Base de conhecimento externa – documentos, PDFs, CSVs, sites, etc.

Como funciona:

* Em vez do modelo depender apenas do conhecimento interno, ele consulta documentos relevantes em uma base externa.

* O processo típico inclui:

* Pré-processamento: dividir documentos em chunks, criar embeddings.

* Armazenamento vetorial: usar FAISS ou outro índice para buscar chunks semanticamente relevantes.

* Consulta: dado um prompt/pergunta, buscar os chunks mais próximos.

* Geração: o LLM gera uma resposta baseada nos chunks recuperados.

Vantagens do RAG:

* Respostas mais precisas e atualizadas.

* Reduz o risco de "alucinação" do modelo.

* Permite lidar com bases de conhecimento muito grandes sem sobrecarregar o modelo.

In [ ]:
from IPython.display import Image, display
# documentação huggingface: https://huggingface.co/learn/cookbook/en/advanced_rag
# documentação langchain: https://python.langchain.com/docs/tutorials/rag/

url = 'https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png'
display(Image(url=url))


In [ ]:
import pandas as pd #Nossa base de dados é um dataset com todos os presidentes da república desde Deodoro da Fonseca até Lula no seu 3º governo. Todos os textos foram raspados da wikipedia
# link: https://pt.wikipedia.org/wiki/Lista_de_presidentes_do_Brasil

caminho_arquivo = 'https://raw.githubusercontent.com/sicdavyd/dados/refs/heads/main/presidentes_brasil_definitivo.csv'

In [ ]:
dados = pd.read_csv(caminho_arquivo)

In [ ]:
dados[:10]

,Unnamed: 0,nome,historia
0,0,Primeira República,Ditadura militar ( 1889 – 1894 )\nOligarquia (...
1,1,Deodoro da Fonseca,Manuel Deodoro da Fonseca ( Alagoas da Lagoa d...
2,2,Floriano Peixoto,"Floriano Vieira Peixoto ( Maceió , 30 de abril..."
3,3,Prudente de Moraes,Prudente José de Morais Barros ( Distrito do V...
4,4,Campos Salles,"Manoel Ferraz de Campos Sales ( Campinas , 1..."
5,5,Rodrigues Alves,Francisco de Paula Rodrigues Alves ( Guarating...
6,6,Affonso Penna,Afonso Augusto Moreira Pena ( Santa Bárbara ...
7,7,Nilo Peçanha,Nilo Procópio Peçanha GCSE ( Campos dos Goytac...
8,8,Hermes da Fonseca,"Hermes Rodrigues da Fonseca ( São Gabriel , 12..."
9,9,Wenceslau Braz,Wenceslau Braz Pereira Gomes ONM ( São Cae...


In [ ]:
from langchain.docstore.document import Document as LangchainDocument # Essa classe representa cada documento, com conteúdo e metadados, é a estrutura padrão de dados p/represenstar pedaços de info. em rag

# cria uma lista de documentos a partir de um DataFrame chamado 'dados'
BASE_DADOS_BRUTA = [
    # P/ cada linha do dataframe, cria um documento do langchain, page_content armazena 'historia', metadata armazena outras
    LangchainDocument(page_content=row["historia"], metadata={"nome": row["nome"]})
    for _, row in dados.iterrows()
]


In [ ]:
BASE_DADOS_BRUTA[11:12]

[Document(metadata={'nome': 'Epitacio Pessôa'}, page_content='Epitácio Lindolfo da Silva Pessoa   ( Umbuzeiro , 23 de maio de 1865 – Petrópolis , 13 de fevereiro de 1942 ) foi um magistrado , diplomata , professor universitário , jurista e político brasileiro , filiado ao Partido Republicano Mineiro . Foi o 11.º presidente do Brasil entre os anos de 1919 a 1922 , tendo o seu governo marcado por revoltas militares que acabariam na Revolução de 1930 , a qual levou Getúlio Vargas ao governo central.\nAnteriormente, foi deputado federal em duas oportunidades, ministro da Justiça , do Supremo Tribunal Federal , procurador-geral da República , senador três vezes, chefe da delegação brasileira junto à Conferência de Versalhes e juiz da então Corte Permanente de Justiça Internacional .\nNasceu na Paraíba , onde os pais morreram de varíola quando tinha sete anos de idade. Foi educado e criado pelo tio, Henrique de Lucena , o Barão de Lucena, então governador de Pernambuco . Era primo em 2º grau

In [ ]:
# Importa o utilitário para dividir textos longos em pedaços menores chunks;
from langchain.text_splitter import RecursiveCharacterTextSplitter

separador_texto = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # número máximo de caracteres em cada chunk (definido arbitrariamente)
    chunk_overlap=150,  # número de caracteres que se sobrepõem entre chunks consecutivos
    add_start_index=True,  # se True, adiciona no metadado o índice inicial do chunk dentro do documento original
    strip_whitespace=True,  # se True, remove espaços em branco no início e fim de cada chunk

)

# lista que armazenará os documentos após a divisão em chunks
documentos_processados = []

# Para cada documento da base de conhecimento original 'BASE_DADOS_BRUTA',
# aplicamos a divisão em chunks e acumulamos os resultados.
for doc in BASE_DADOS_BRUTA:
    documentos_processados += separador_texto.split_documents([doc])

### Ao rodar a célula abaixo provavelmente aparecerá uma mensagem pedindo para liberar acesso ao HF TOKEN, permita!

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch
#from langchain_huggingface import HuggingFaceEmbeddings

# detecta se há GPU disponível cuda, caso contrário usa cpu
dispositivo = "cuda" if torch.cuda.is_available() else "cpu"

# criamos o modelo de embeddings multilíngue (aceita ptbr)
# modelo escolhido é leve e eficiente para comparação semântica de frases.
modelo_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # modelo usado
    model_kwargs={"device": dispositivo},         # executa na GPU se disponível
    encode_kwargs={"normalize_embeddings": True},  # normaliza vetores para similaridade por cosseno
)

# Criamos a base vetorial usando FAISS
# Cada documento previamente dividido (documentos_processados) é convertido em embedding
# e armazenado em uma estrutura de busca eficiente.
vector_db = FAISS.from_documents( #FAISS indexa os embeddings gerados no seu banco de dados vetorial
    documentos_processados,  # conjunto de chunks processados
    modelo_embedding,        # modelo de embeddings em português
)


/tmp/ipython-input-3185217437.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  modelo_embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
docs = vector_db.docstore._dict  # dicionário de {chave: Document}
for k, v in docs.items():
    print("ID:", k)
    print("Texto:", v.page_content[:200])  # mostra só os 200 primeiros caracteres
    print("Metadados:", v.metadata)
    print("-----")


ID: 475e6cff-1c57-4813-b101-52fe118ac358
Texto: Ditadura militar ( 1889 – 1894 )
Oligarquia ( 1894 – 1930 )
A Primeira República Brasileira , também conhecida como República Velha ou República das Oligarquias , é o período da história do Brasil que
Metadados: {'nome': 'Primeira República', 'start_index': 0}
-----
ID: f7e3af5f-7191-4e6f-a635-58bfddcfd657
Texto: O segundo período ficou conhecido como " República Oligárquica ", e se estendeu de 1894 até à Revolução de 1930 . Caracterizou-se por dar maior poder para as elites regionais, em especial do sudeste d
Metadados: {'nome': 'Primeira República', 'start_index': 993}
-----
ID: 276b9110-08d8-4b14-8034-b6d674ca40e8
Texto: O primeiro partido republicano no Brasil foi o Partido Republicano Paulista (PRP), criado na Convenção de Itu , em 1873. O PRP era um partido legalizado, apesar de o Brasil ser uma monarquia . O PRP c
Metadados: {'nome': 'Primeira República', 'start_index': 1702}
-----
ID: 8a3f1168-6363-491f-a21e-e838ee7347bf
Texto: O 

In [ ]:
#Visualizando os vetores gerados
vetores = vector_db.index.reconstruct_n(0, vector_db.index.ntotal)

print("Primeiro vetor:", vetores[0][:15])

Primeiro vetor: [-0.08508681  0.00382108  0.03011023 -0.01475741  0.04842558  0.02741401
  0.00877231  0.09592881  0.10483263  0.0527548   0.01591914 -0.01125728
 -0.00253376  0.03232085 -0.06381124]


In [ ]:
# definir a pergunta que queremos testar
pergunta = "Juscelino Kubitschek" # Nesse teste a resposta correta está no chunk 6

''' buscar os chunks mais relevantes na base vetorial
    k=8 retorna os 8 chunks mais próximos da pergunta
    você pode testar com valores diferentes de k para ver como muda o resultado '''
resultado = vector_db.similarity_search(pergunta, k=8)

for i, doc in enumerate(resultado):
    print(f"Chunk {i+1}:")
    print(doc.page_content[:500])
    print(f'--' * 250)


Chunk 1:
A vida e carreira política de Juscelino Kubitschek foi tema de muitos livros e, de 3 de janeiro até 24 de março de 2006, foi contada através de uma minissérie da Rede Globo intitulada " JK ".  
Juscelino Kubitschek foi retratado como personagem no cinema, teatro e na televisão, interpretado por José de Abreu no filme " Bela Noite para Voar " (2005), e José Wilker e Wagner Moura na minissérie de televisão " JK " em 2006.   No teatro foi interpretado pelo ator John Vaz no espetáculo "JK 1902 - 200
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Abaixo você deve inserir sua Google API Key entre aspas, simples ou duplas.

In [ ]:
import os
from langchain.chat_models import init_chat_model

# Substitua 'SUA_CHAVE' pela chave real
os.environ["GOOGLE_API_KEY"] = 'SUA_SENHA_AQUI'
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain.chains import RetrievalQA

# criar o retriever a partir da base FAISS
# ele vai buscar os chunks mais relevantes na base vetorial
retriever = vector_db.as_retriever(
    search_type="mmr",  # tipo de busca: "similarity" (mais comum - similaridade do cosseno) ou "mmr" (para maior diversidade) brinque com isso!
    search_kwargs={"k": 5}     # número de chunks que serão retornados
)

# montar o pipeline RAG usando o LLM (ex.: Gemini) e o retriever
# o RetrievalQA conecta a busca de chunks com a geração da resposta pelo modelo, essencial
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                # modelo de linguagem que vai gerar a resposta
    retriever=retriever,    # componente que busca no FAISS
    chain_type="stuff"      # "stuff" concatena todos os chunks encontrados de forma simples
)

# testar o pipeline com uma pergunta
pergunta = "Juscelino Kubitschek"
resposta = qa_chain.run(pergunta)

# exibir resultado
print("Pergunta:", pergunta)
print("Resposta:", resposta)


Pergunta: Juscelino Kubitschek
Resposta: Juscelino Kubitschek foi uma figura proeminente na política brasileira, com sua vida e carreira sendo tema de diversas obras e homenagens:

*   **Representações na Mídia:**
    *   Sua vida foi contada na minissérie "JK" da Rede Globo, exibida de janeiro a março de 2006.
    *   No cinema, foi interpretado por José de Abreu no filme "Bela Noite para Voar" (2005).
    *   Na minissérie "JK", foi interpretado por José Wilker e Wagner Moura.
    *   No teatro, John Vaz o interpretou na peça "JK 1902 - 2002".
    *   Foi tema de documentários dirigidos por Sílvio Tendler: "Os Anos JK - Uma Trajetória Política" (1980) e "JK – O Menino que Sonhou um País" (2002).

*   **Homenagens e Legado:**
    *   Sua efígie foi impressa nas notas de Cz$ 100,00 (cem cruzados) em 1986.
    *   Teve sua efígie cunhada no verso das moedas de um real, lançadas em 2002, em comemoração ao centenário de seu nascimento.
    *   Diversas vias e locais foram nomeados em sua 

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# prompt inicial (instrução de como o modelo deve responder)
prompt_inicial = PromptTemplate(
    template="""
Você é um historiador acadêmico especialista em história do Brasil, conhecido por sua precisão e objetividade. Sua tarefa é responder perguntas baseando-se *exclusivamente* nas informações fornecidas no contexto. Siga as instruções e os exemplos de comportamento abaixo.

**Instruções rigorosas:**
1.  **Atenha-se ao Contexto:** Extraia e sintetize a resposta apenas do texto fornecido em "Contexto". Não utilize conhecimento prévio ou informações externas.
2.  **Seja Direto e Conciso:** Formule uma resposta clara, objetiva e sem rodeios.
3.  **Regra de Recusa:** Se a resposta não estiver no contexto, responda exatamente: "A informação para responder a esta pergunta não foi encontrada no contexto fornecido."

---
**Exemplo 1 (Resposta Encontrada):**

**Contexto:** Durante a Era Vargas, especificamente no período do Estado Novo, o presidente Getúlio Vargas promoveu uma significativa centralização do poder. Uma de suas ações mais notáveis foi a criação da Consolidação das Leis do Trabalho (CLT) em 1º de maio de 1943, que unificou a legislação trabalhista no país.
**Pergunta:** Qual foi a principal legislação trabalhista criada por Getúlio Vargas e em que ano?
**Resposta:** Segundo o contexto, a principal legislação trabalhista criada por Getúlio Vargas foi a Consolidação das Leis do Trabalho (CLT), em 1943.

---
**Exemplo 2 (Resposta NÃO Encontrada):**

**Contexto:** O governo de Juscelino Kubitschek (1956-1961) foi marcado pelo seu ambicioso Plano de Metas, cujo lema era "cinquenta anos em cinco". O projeto mais emblemático de seu mandato foi a construção de uma nova capital federal, Brasília, inaugurada em 21 de abril de 1960.
**Pergunta:** Qual era o nome da esposa de Juscelino Kubitschek?
**Resposta:** A informação para responder a esta pergunta não foi encontrada no contexto fornecido.

---
**Exemplo 3 (Síntese de Informação):**

**Contexto:** A presidência de Fernando Henrique Cardoso (FHC) teve como pilar a estabilização econômica do país, consolidando o Plano Real, que controlou a hiperinflação. Além da política monetária, seu governo também implementou um vasto programa de privatizações de empresas estatais em setores como telecomunicações e mineração.
**Pergunta:** Quais foram as duas principais frentes de atuação do governo FHC mencionadas no texto?
**Resposta:** De acordo com o texto, as duas principais frentes de atuação do governo FHC foram a consolidação do Plano Real para controlar a inflação e a implementação de um programa de privatizações de empresas estatais.
---

**Sua Tarefa:**

**Contexto:**
{context}

**Pergunta:**
{question}

**Resposta:**
""",
    input_variables=["context", "question"]
)

## Vamos fazer o mesmo processo feito anteriormente, porém, com Prompt que utiliza a técnica few-shot

In [ ]:
# criar o retriever a partir da base FAISS
retriever_ = vector_db.as_retriever(
    search_type="similarity", #afeta como os chunks são buscados; outra opção: mmr
    search_kwargs={"k": 30} #top k entidades retornadas pelo retriever
)

# montar o pipeline RAG com prompt inicial
qa_chain_ = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_,
    chain_type="stuff", # define como os chunks são combinados antes do modelo gerar a resposta, "stuff" concatena todos os chunks encontrados de forma simples
    chain_type_kwargs={"prompt": prompt_inicial},  # parametros extras, como: prompt_template

)

In [ ]:
# testar o pipeline
pergunta_ = "Juscelino Kubitschek"
resposta_ = qa_chain_.run(pergunta_)

print("Pergunta:", pergunta_)
print("Resposta:", resposta_)

Pergunta: Juscelino Kubitschek
Resposta: Juscelino Kubitschek de Oliveira, também conhecido como JK, nasceu em Diamantina, Minas Gerais, em 12 de setembro de 1902, e faleceu em Resende em 22 de agosto de 1976. Ele foi médico, oficial da Polícia Militar mineira e o 21º Presidente do Brasil entre 1956 e 1961. Formou-se em medicina pela Universidade Federal de Minas Gerais (UFMG) em 1927 e especializou-se em urologia em Paris em 1930. Casado com Sarah Lemos, teve uma filha, Márcia, e adotou Maria Estela. Sua mãe, Júlia Kubitschek, era professora primária de ascendência checa, e seu pai, João César de Oliveira, foi caixeiro-viajante e delegado de polícia. Seu bisavô materno, Jan Nepomuk Kubíček, chegou ao Brasil por volta de 1835 da Boêmia.

Como presidente, seu governo foi caracterizado pelo desenvolvimentismo e pelo Plano de Metas, com o lema "50 anos em 5", culminando na inauguração de Brasília em 1960. Ele promoveu a abertura da economia brasileira ao capital estrangeiro, isentando de 

In [ ]:
resposta_

'Juscelino Kubitschek de Oliveira, também conhecido como JK, nasceu em Diamantina, Minas Gerais, em 12 de setembro de 1902, e faleceu em Resende em 22 de agosto de 1976. Ele foi médico, oficial da Polícia Militar mineira e o 21º Presidente do Brasil entre 1956 e 1961. Formou-se em medicina pela Universidade Federal de Minas Gerais (UFMG) em 1927 e especializou-se em urologia em Paris em 1930. Casado com Sarah Lemos, teve uma filha, Márcia, e adotou Maria Estela. Sua mãe, Júlia Kubitschek, era professora primária de ascendência checa, e seu pai, João César de Oliveira, foi caixeiro-viajante e delegado de polícia. Seu bisavô materno, Jan Nepomuk Kubíček, chegou ao Brasil por volta de 1835 da Boêmia.\n\nComo presidente, seu governo foi caracterizado pelo desenvolvimentismo e pelo Plano de Metas, com o lema "50 anos em 5", culminando na inauguração de Brasília em 1960. Ele promoveu a abertura da economia brasileira ao capital estrangeiro, isentando de impostos de importação máquinas e equi